In [1]:
import json
import requests
from statistics import mean 

## Data Cleaning and Preparation

In [2]:
def return_url_params(var_url,player_id,start_date,year,oponent_team_id,venue_id,matchtype=None):
    year_wise_url = var_url+"api/year_wise_playermatchstats/"
    year_wise_params = {'player_id':player_id,'start_date':start_date,'year':year,'matchtype':matchtype}
    opponent_wise_url = var_url + 'api/opponent_wise_playermatchstats/'
    opponent_wise_params = {'player_id':player_id,'team_id':oponent_team_id,'start_date':start_date,'matchtype':matchtype}
    venue_wise_url = var_url + "api/venue_wise_playermatchstats/"
    venue_wise_params = {'player_id':player_id,'venue_id':venue_id,'start_date':start_date,'matchtype':matchtype}
    overall_url = var_url+'api/playermatchstats/'
    overall_params = {'player_id':player_id,'start_date':start_date}
    url_parms = {
        'v_u':venue_wise_url,
        'v_p':venue_wise_params,
        'o_u':opponent_wise_url,
        'o_p':opponent_wise_params,
        'y_u':year_wise_url,
        'y_p':year_wise_params,
        'ov_u':overall_url,
        'ov_p':overall_params
    }
    return url_parms

In [3]:
local_var = 'http://localhost:8000/'
global_var = 'https://win11.neolen.com/'

In [4]:
def get_stats(url,params):
    response = requests.request("GET", url, params = params)
    json_data = json.loads(response.text)
    return json_data

def return_int(d):
    try:
        d = int(d)
        return d
    except:
        return 0

In [5]:
def return_stats(json_data):     
    match_count = 0
    bowl_avg_list = []
    order_list = []
    bat_inning_count = 0
    bowl_inning_count = 0
    bat_avg_list = []
    list_50 = []
    list_100 = []
    list_5w = []
    list_3w = []
    for m in json_data:
        scorecard = m.get('scorecard',[])
        match_count += 1
        bowling_avg = 0 
        batting_avg = 0
        for s in scorecard:
            bat = s.get('match_batting_inning')
            bowl = s.get('match_bowling_inning')
            
            bat = bat[0] if len(bat) != 0 else None
                
            if bat:
                
                order = bat.get('order')
                run = return_int(bat.get('runs'))
                f_w = bat.get('fall_of_wicket_over')
                how_out = bat.get('how_out')

                #calculate batting average
                if f_w != '' and how_out != '':
                    bat_avg_list.append(float(run))
                elif how_out != "not out" and how_out != 'retired hurt':
                    bat_avg_list.append(float(run))
                else:
                    bat_avg_list.append(0.0)
                #end

                #calculate 50
                list_50.append(1 if run >= 50 and run < 100 else 0)
                #end

                #calculate 100
                list_100.append(1 if run>=100 else 0)
                #end
                
                bat_inning_count+=1
                order_list.append(order)

            bowl = bowl[0] if len(bowl) != 0 else None

            if bowl:
                
                run_conceded = return_int(bowl.get('run_conceded',0))
                wickets = return_int(bowl.get('wickets',0))
                
                #calculate bowling average
                if wickets != 0:
                    bowling_avg = run_conceded/wickets
                    bowl_avg_list.append(bowling_avg)
                else:
                    bowl_avg_list.append(0.0)
                #end
                
                #5 and 3 w
                list_3w.append(1 if wickets >= 3 and wickets < 5 else 0)
                list_5w.append(1 if wickets >= 5 else 0)
                #end
                
                bowl_inning_count+=1

#     print(match_count,bat_avg_list,bowl_avg_list,list_100,list_50,list_3w,list_5w,order_list,sep = '\n')
    bat_data = {
        'total_match':match_count,
        'innings':bat_inning_count,
        'avg':mean(bat_avg_list) if len(bat_avg_list) != 0 else 0,
        '50': sum(list_50),
        'order':mean(order_list) if len(order_list) != 0 else 0,
        '100':sum(list_100),
    }
    bowl_data = {
        'total_match':match_count,
        'innings': bowl_inning_count,
        'avg': mean(bowl_avg_list) if len(bowl_avg_list) != 0 else 0,
        '5w': sum(list_5w),
        '3w': sum(list_3w),
    }
    return bat_data , bowl_data

In [6]:
def batting_score(data,MAX_BATTING_AVG=250,MAX_50_100=30.0): #20*49+5*96
    B = data['avg']/MAX_BATTING_AVG
    C = (20*data['100'] + 5*data['50'])/data['innings']
    D = C/MAX_50_100
    E = 0.4*D + 0.6*B
    return E
def bowling_score(data,MAX_BOWL_AVG=150,MAX_5_3=10.0):
    H = data['avg']/MAX_BOWL_AVG
    F = (20*data['5w'] + 5*data['3w'])/data['innings'] if data['innings'] != 0 else 0
    G = F/MAX_5_3
    I = 0.4*G + 0.6*H
    return I

In [7]:
def total_batting_score(data,player_stats):
    overall = player_stats['overall'][0]
    A = overall['innings']/overall['total_match']
    E = data['overall']
    R = data['venue']
    P = data['year']
    Q = data['oponent']
    L = data['last_5']
    ans = A*((0.25*E) + (0.15*P) + (0.1*(Q+R))+(0.5*L))
    return ans
def total_bowling_score(data,player_stats):
    overall = player_stats['overall'][1]
    V = overall['innings']/overall['total_match']
    I = data['overall']
    S = data['year']
    T = data['oponent']
    U = data['venue']
    L1 = data['last_5']
    ans = V*((0.25*I) +(0.15*S) + (0.1 *
    (T+U) )+(0.5*L1))
    return ans

In [8]:
url_match = 'api/peryearmatch/'
matches_19 = get_stats(local_var+url_match,{'year':2019})

In [9]:
for m in matches_19['results']:
    url_match_player = 'api/permatchplayer/'
    players_team1_match = get_stats(local_var+url_match_player,{'match_id':m['id'],'team_id':m['team1']})['results'][0]['players']
    players_team2_match = get_stats(local_var+url_match_player,{'match_id':m['id'],'team_id':m['team2']})['results'][0]['players']
    print(players_team1_match,players_team2_match,sep='\n')
    bat_score_player_id = {}
    bow_score_player_id = {}
    all_rounder_score_player = set()
    for p in players_team1_match:
        batting_score_id = {}
        bowling_score_id = {}
        player_all_stats = return_url_params(local_var,p,m['start_date'],2019,m['team2'],m['venue'])
        player_stats = {
            'year': return_stats(get_stats(player_all_stats['y_u'],player_all_stats['y_p'])['result']),
            'oponent': return_stats(get_stats(player_all_stats['o_u'],player_all_stats['o_p'])['result']),
            'venue': return_stats(get_stats(player_all_stats['v_u'],player_all_stats['v_p'])['result']),
            'overall': return_stats(get_stats(player_all_stats['ov_u'],player_all_stats['ov_p'])['result']),
            'last_5': return_stats(get_stats(player_all_stats['ov_u'],player_all_stats['ov_p'])['result'][:5])
        }
        batting_score_id = {
            'overall' :batting_score(player_stats['overall'][0]),
             'venue'  :batting_score(player_stats['venue'][0]),
            'oponent':batting_score(player_stats['oponent'][0]),
             'year'   :batting_score(player_stats['year'][0]),
            'last_5': batting_score(player_stats['last_5'][0])
        }
        bowling_score_id = {
            'overall' :bowling_score(player_stats['overall'][1]),
             'venue'  :bowling_score(player_stats['venue'][1]),
            'oponent':bowling_score(player_stats['oponent'][1]),
             'year'   :bowling_score(player_stats['year'][1]),
            'last_5': bowling_score(player_stats['last_5'][1])
        }
        bat_score_player_id[p] = total_batting_score(batting_score_id,player_stats)
        bow_score_player_id[p] = total_bowling_score(bowling_score_id,player_stats)
    print(bat_score_player_id,bow_score_player_id,sep='\n')
    #calculate the all rounder
    for p in players_team1_match:
        avg = (bat_score_player_id[p]+bow_score_player_id[p])/2
        all_rounder_score_player.add((avg,p))
    print(all_rounder_score_player)
    break

[478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488]
[489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499]
{478: 0.1541656341079853, 479: 0.10928468229166667, 480: 0.12600119540229884, 481: 0.10789521840539289, 482: 0.11361593856270974, 483: 0.0897073066347082, 484: 0.06851834972677595, 485: 0.04189020361247947, 486: 0.007457391304347826, 487: 0.0, 488: 0.006945588428665351}
{478: 0.0, 479: 0.0, 480: 0.005597290640394088, 481: 0.0007692650334075724, 482: 0.03444896121883656, 483: 0.11357366576796285, 484: 0.034694689518132135, 485: 0.08112756049606776, 486: 0.248528115942029, 487: 0.06344, 488: 0.18504008547008552}
{(0.07403244989077315, 482), (0.06579924302134646, 480), (0.1279927536231884, 486), (0.05433224171940023, 481), (0.07708281705399266, 478), (0.054642341145833336, 479), (0.05160651962245404, 484), (0.061508882054273614, 485), (0.03172, 487), (0.09599283694937544, 488), (0.10164048620133553, 483)}


## Statistical Model Implementation